In [109]:
import pandas as pd
import numpy as np
import time
import pyarrow.parquet as pq
import scipy
import bisect
import sklearn.metrics as m
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
import implicit
import faiss
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
import pycaret.classification as pc

In [110]:
#чтение, вывод инфы
data = pq.read_table('data/part-00000-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet')
#display(pd.DataFrame([(z.name, z.type) for z in data.schema], columns = [['field', 'type']]))
print(len(data))
targets = pq.read_table('data/public_train.pqt')
display(pd.DataFrame([(z.name, z.type) for z in targets.schema], columns = [['field', 'type']]))
print(len(targets))

32638709


,field,type
0,age,double
1,is_male,string
2,user_id,int64
3,__index_level_0__,int64


270000


In [111]:
data.select(['cpe_type_cd']).to_pandas()['cpe_type_cd'].value_counts()

smartphone    32630468
plain             3995
tablet            3204
phablet           1042
Name: cpe_type_cd, dtype: int64

In [156]:
data_agg = data.select(['user_id', 'url_host', 'request_cnt']).group_by(['user_id', 'url_host']).aggregate([('request_cnt', "sum")])

In [ ]:
#первый элемент url_set - tes.reactor.cc
url_set = set(data_agg.select(['url_host']).to_pandas()['url_host'])

#создаем словарь, где ключ - имя сайта, а значение - номер в сеттере
url_dict = {url: idurl for url, idurl in zip(url_set, range(len(url_set)))}
print(url_dict['tes.reactor.cc'])

#первый элемент usr_set - 131073
usr_set = set(data_agg.select(['user_id']).to_pandas()['user_id'])
#словарь, где ключ - id пользователя, а значение - номер в сеттере
usr_dict = {usr: user_id for usr, user_id in zip(usr_set, range(len(usr_set)))}
print(usr_dict[131073])

Делаем странные вещи

In [ ]:
values = np.array(data_agg.select(['request_cnt_sum']).to_pandas()['request_cnt_sum'])

#создаем массив, где каждой строке колонки user_id присваивается номер в сеттере
print(usr_dict[45098])
display(data_agg.select(['user_id']).to_pandas()['user_id'].head())
display(data_agg.select(['user_id']).to_pandas()['user_id'].map(usr_dict).head())
rows = np.array(data_agg.select(['user_id']).to_pandas()['user_id'].map(usr_dict))
print(rows,'\n\n\n\n')

#создаем массив, где каждой строке колонки url_host присваивается номер в сеттере
print(url_dict['ad.adriver.ru'])
display(data_agg.select(['url_host']).to_pandas()['url_host'].head())
display(data_agg.select(['url_host']).to_pandas()['url_host'].map(url_dict).head())
cols = np.array(data_agg.select(['url_host']).to_pandas()['url_host'].map(url_dict))
print(cols)

#создаем матрицу, которая хранит сеттреский id, сеттерский url и сумму заходов с этого id на этот url
mat = scipy.sparse.coo_matrix((values, (rows, cols)))
print(f'Для юзера с номером в сеттере 15651 и url в сеттере 28992 вот такая сумма заходов на юрл:{mat.tocsr()[15651, 28992]}')
display(data_agg.to_pandas().loc[(data_agg.to_pandas()['user_id'] == 45098) & (data_agg.to_pandas()['url_host'] == 'ad.adriver.ru')])

In [ ]:
als = implicit.approximate_als.FaissAlternatingLeastSquares(factors = 50,
                                                            iterations = 30,
                                                            use_gpu = False,
                                                            calculate_training_loss = False,
                                                            regularization = 0.1)
als.fit(mat)
u_factors = als.model.user_factors
print(u_factors)

типа подготовка данных

In [ ]:
%%time
inv_usr_map = {v: k for k, v in usr_dict.items()}
usr_emb = pd.DataFrame(u_factors)
usr_emb['user_id'] = usr_emb.index.map(inv_usr_map)
usr_targets = targets.to_pandas()
df = usr_targets.merge(usr_emb, how = 'inner', on = ['user_id'])
df = df[df['is_male'] != 'NA']
df = df.dropna()
df['is_male'] = df['is_male'].map(int)
df['is_male'].value_counts()

каким-то боком еще и кетбуст

In [ ]:
%%time
x_train, x_test, y_train, y_test = train_test_split( \
    df.drop(['user_id', 'age', 'is_male'], axis = 1), df['is_male'], test_size = 0.33, random_state = SPLIT_SEED)
clf = CatBoostClassifier()
clf.fit(x_train, y_train, verbose = False)
print(f'GINI по полу {2 * m.roc_auc_score(y_test, clf.predict_proba(x_test)[:,1]) - 1:2.3f}')

С возрастом у них более менее адекватно, есть крутая штука classification_report